In [ ]:
from datasets import load_dataset
ds = load_dataset("argilla/llama-2-banking-fine-tune",split="train")

In [ ]:
len(ds)

In [ ]:
ds[0]

In [5]:
ds_formatted = [
    {"messages": [
        {"role": "system", "content": "You are a customer service representative from Bank of America. Please reply customer requests using polite and respectful language."},
        {'role': 'user', 'content': x["request"]},
        {'role': 'assistant', 'content': x["response-1"]}]} for x in ds
]

In [ ]:
ds_formatted[0]

In [7]:
import random
random.shuffle(ds_formatted)

In [8]:
ds_train = ds_formatted[:80]
ds_val = ds_formatted[80:]

In [9]:
import json

with open('train.jsonl', 'w') as f:
    for line in ds_train:
        json.dump(line, f)
        f.write('\n')

with open('val.jsonl', 'w') as f:
    for line in ds_val:
        json.dump(line, f)
        f.write('\n')

In [ ]:
ls

# Upload data

In [13]:
import os
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv()

API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key= API_KEY )

In [ ]:
from pathlib import Path
# Upload training data
train = client.files.create(
  file=Path('train.jsonl'),
  purpose='fine-tune'
)
train

In [ ]:
train_id = train.id
train_id

In [ ]:
from pathlib import Path
# Upload validation data
val = client.files.create(
  file=Path('val.jsonl'),
  purpose='fine-tune'
)
val

In [ ]:
val_id = val.id
val_id

In [ ]:
client.files.list()

# Fine-tuning

In [ ]:
# Create a fine-tuned model
response = client.fine_tuning.jobs.create(
    model = "gpt-3.5-turbo",
    training_file=train_id,
    validation_file=val_id,
    suffix="gpt-bank-custom"
    )
response

In [ ]:
job_id = response.id
job_id

In [ ]:
# Retrieve the state of a fine-tune
response =  client.fine_tuning.jobs.retrieve(job_id)
response

In [ ]:
fine_tune_model_id = response.fine_tuned_model
fine_tune_model_id

# Inference

In [ ]:
test_messages = [
    {"role": "system", "content": "You are a customer service representative from Bank of America. Please reply customer requests using polite and respectful language."},
    {"role": "user", "content": "My card is stolen!"}
  ]

In [ ]:
completion = client.chat.completions.create(
  model=fine_tune_model_id,
  messages=test_messages,
  temperature=0
)


In [ ]:
print(completion.choices[0].message.content)

In [ ]:
completion1 = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=test_messages
)

In [ ]:
print(completion1.choices[0].message.content)